In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

# import everything from models: files, tags, etc
from models import *

In [3]:
settings.set_up()
r = main.Main()

[(0, 29), (6, 29), (12, 29), (14, 29), (9, 29), (4, 29), (6, 29), (1, 29), (7, 29), (18, 29), (9, 29), (20, 29), (8, 29), (19, 29), (11, 29), (3, 29), (11, 29), (17, 29), (6, 29), (13, 29), (8, 29), (5, 29), (11, 29), (1, 29), (6, 29), (8, 29), (3, 29), (2, 29), (12, 29), (11, 29), (12, 29), (2, 29), (11, 29), (8, 29), (1, 29), (9, 29), (10, 29), (9, 29), (0, 29), (10, 29), (5, 29), (16, 29), (8, 29), (10, 29), (14, 29), (17, 29), (10, 29), (11, 29), (12, 29), (4, 29), (11, 29), (21, 29), (31, 29), (11, 29), (13, 29), (6, 29), (12, 29), (5, 29), (8, 29), (30, 29), (6, 29), (11, 29), (3, 29), (29, 29), (27, 29), (12, 29), (32, 29), (9, 29), (9, 29), (12, 29), (12, 29), (18, 29), (12, 29), (22, 29), (16, 29)]


# OCRed Document Structure -> Flat files + Company name JSON

In [28]:
import os
from models.tagged_file import TFile
import shutil

home = os.path.expanduser("~")
ocr_dir = os.path.join(home, "Dropbox/Data Scrape of VC/OCRed Documents")
output_dir = os.path.join(home, "Dropbox/Data Scrape of VC/txt_flat_files")

coi = "Certificates of Incorporation"
unlabeled_tfiles = []
name_to_company = {}
i = 100
for d in os.listdir(ocr_dir):
    company_path = os.path.join(ocr_dir, d, coi)
    if not os.path.isdir(company_path):
        print "!!! %s doesn't have articles of incorporation" % company_path
        continue
        
    company_name = d
    for article in os.listdir(company_path):
        article_path = os.path.join(company_path, article)
        
        if os.path.isfile(article_path):
            unlabeled_tfiles += [TFile(article_path, i)]
            i += 1
            name_to_company[article_path] = company_name
            shutil.copy(article_path, output_dir)


!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/Accruent LLC/Certificates of Incorporation doesn't have articles of incorporation
!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/Arroweye Solutions, Inc/Certificates of Incorporation doesn't have articles of incorporation
!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/Ascenta Therapeutics, Inc/Certificates of Incorporation doesn't have articles of incorporation
!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/Azimuth Systems, Inc/Certificates of Incorporation doesn't have articles of incorporation
!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/BG Medicine, Inc/Certificates of Incorporation doesn't have articles of incorporation
!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/BiggerBoat/Certificates of Incorporation doesn't have articles of incorporation
!!! /Users/dima/Dropbox/Data Scrape of VC/OCRed Documents/Caden Biosciences Inc/Certificates of Incorporation doesn't have art

In [16]:
len(unlabeled_tfiles)

10269

In [93]:
import json

nametail_to_company = {}
for key, item in name_to_company.iteritems():
    _, key_tail = os.path.split(key)
    nametail_to_company[key_tail] = item
print nametail_to_company.keys()[:1]
    
with open("name_to_company.json",'w') as out_f:
    json.dump(name_to_company, out_f)
    
with open("nametail_to_company.json", 'w') as out_f:
    json.dump(nametail_to_company, out_f)


['Marathon Technologies 02112010.txt']


In [26]:
output_dir = os.path.join(home, "Dropbox/Data Scrape of VC/txt_flat_files")

In [128]:
small_train_set = unlabeled_tfiles[:300]
rest_of_set = unlabeled_tfiles[300:]


file_set = small_train_set #unlabeled_tfiles, rest_of_set

In [129]:
for f in file_set:
    if len(f.numbers) > 0:
        f.predictions = zip(r.cl.predict_class_for_each_number(f),f.numbers)
    else:
        f.predictions = []


## Iterate over the new files, see if they belong to previous. If not, then add them to DB

In [130]:
from pymongo import MongoClient
client = MongoClient('localhost', 3001)
client.database_names()
db = client.meteor
from pymongo import collection
from pymongo import InsertOne

## Get the databases. Add create=True to create if needed
docs = collection.Collection(db, 'docs')
tags = collection.Collection(db, 'tags')

In [137]:
import os
import random
import time

requests = []
with open("existing_docs.json") as exist_file:
    existing_files = set(json.load(exist_file))
skipped = 0

def generate_string_id(i=None):
    if i:
        return str(int(time.time() * 1000)) + str(i) + str(int(random.random() * 1000))
    else:
        return str(int(time.time() * 1000)) + str(int(random.random() * 10000))

# for f in unlabeled_tfiles:
# for f in rest_of_set:
for f in file_set:
    
    doc_repr = {}
    doc_repr['contents'] = f.raw_clean
    doc_repr['filename'] = f.filename
    head, tail = os.path.split(f.filename)
    if tail in existing_files:
        skipped += 1
        docs.update({"filename_tail": tail}, {"$set": {"company": name_to_company[f.filename]}})
        continue
    
    doc_repr['filename_tail'] = tail
    doc_repr['old_id'] = f.id
    doc_repr['batch'] = 2
    doc_repr['company'] = name_to_company[f.filename]
    doc_repr['tags'] = []
    doc_repr['reviewed'] = False
    i = 0
    for prediction, number in f.predictions:
        
        update_doc = True
        rand_forrest_prediction = prediction[1]
        
        translation = {'CS': "Common Shares", 'PS': "Preferred Shares", 'TS': 'Total Shares'}
        description = translation.get(rand_forrest_prediction, None)
        text = number.match
        tag_repr = {'pos_start': number.pos - len(text), 'pos_end': number.pos, 'text': text,
                   'needs_approval': False, 'document_id': doc['_id'], 'id': generate_string_id(i)}
        if description:
            tag_repr['description'] = description
            tag_repr['needs_approval'] = True
        
        doc_repr['tags'].append(tag_repr)
        i += 1
    
    requests.append(InsertOne(doc_repr))
print "Skipped %i in total" %skipped
print "Will write %i new docs" % len(requests)

Skipped 3 in total
Will write 297 new docs


In [138]:
# add documents to the collection
docs.bulk_write(requests)


In [124]:
tag_requests = []
skipped = 0

for i, f in enumerate(file_set):
    if (i%1000 == 0):
        print i
    head, tail = os.path.split(f.filename)
    if tail in existing_files:
        skipped += 1
        continue
    doc = docs.find_one({"old_id": f.id})
    update_doc = False
    for prediction, number in f.predictions:
        update_doc = True
        rand_forrest_prediction = prediction[1]
        
        translation = {'CS': "Common Shares", 'PS': "Preferred Shares", 'TS': 'Total Shares'}
        description = translation.get(rand_forrest_prediction, None)
        text = number.match
        tag_repr = {'pos_start': number.pos - len(text), 'pos_end': number.pos, 'text': text, 'human_approved': False,
                   'hide': False, 'human_created': False, 'document_id': doc['_id']}
        if description:
            tag_repr['description'] = description
        
        tag_requests.append(InsertOne(tag_repr))
#     if update_doc:
#         docs.update_one({"old_id": f.id}, {"$set": {""}})
print "Will insert %i new tags" % len(tag_requests)


0
Will insert 4725 new tags


In [125]:
tags.bulk_write(tag_requests)

In [126]:
print tags.find({}).count()
print docs.find({}).count()



5543
519
